In [ ]:
# Obrigatório para testes no Google Colab, senão ignorar
from google.colab import userdata

github_token = userdata.get('GITHUB_TOKEN')
!git clone https://{github_token}@github.com/erickrribeiro/ufam_nlp_01_2025.git
!mv ./ufam_nlp_01_2025/scripts/** ./
!cp ./ufam_nlp_01_2025/models/* ./
!unzip ./mistral-sql-lora-v3.zip -d ./
!unzip ./mistral-sql-lora-v4.zip -d ./

In [ ]:
# !rm -rf ./ufam_nlp_01_2025
# !rm -rf ./spider
# !rm -rf mistral-sql-lora-*
# !rm *.py
# !rm *.ipynb
# !rm *.csv
# !rm *.zip

In [5]:
!cp ../custom_metrics/accuracy_metrics.py ../

In [ ]:
!pip install -U gdown==5.2.0 \
                transformers==4.52.4 \
                trl==0.18.2 \
                peft==0.15.2 \
                datasets==3.6.0 \
                accelerate==1.8.0 \
                huggingface_hub==0.33.0 \
                fsspec==2025.3.0 \
                deepeval==3.1.6 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import datasets, transformers, trl, peft, accelerate, huggingface_hub, fsspec, deepeval

print("datasets:", datasets.__version__)
print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("trl:", trl.__version__)
print("accelerate:", accelerate.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("fsspec:", fsspec.__version__)
print("deepeval:", deepeval.__version__)

datasets: 3.6.0
transformers: 4.52.4
peft: 0.15.2
trl: 0.18.2
accelerate: 1.8.0
huggingface_hub: 0.33.0
fsspec: 2025.3.0
deepeval: 3.1.6


# Fase 4: Análise Quantitativa de Regressão de Capacidade



- 4.1. Metodologia de Avaliação MMLU: Este notebook avalia o modelo base e os modelos fine-tuned na suíte de 150 questões do [MMLU](https://github.com/hendrycks/test)(Massive Multitask Language Understanding). A avaliação será feita em modo 4-shot para cada uma das quatro opções de múltipla escolha.
- 4.2. Cálculo de Acurácia: A métrica utilizado será a acurácia de resposta correta.
- 4.3. Análise de Regressão: Será calculado a variação percentual de acurácia entre o modelo base e o dois modelos fine-tuned. Esta análise será reportada de forma agregada e também por categoria (STEM, Humanidades, Ciências Sociais) para identificar se a regressão de capacidade afeta domínios de forma heterogênea.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from utils import set_seed
from custom_model import Mistral7B


set_seed(42)

In [ ]:
# Testando a classe Mistral7B implementado no arquivo ./custom_model

# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# baseline = Mistral7B(model=model, tokenizer=tokenizer)
# print(baseline("Write me a joke"))

In [ ]:
import pandas as pd
import torch
from utils import run_mmlu


model_name = "mistralai/Mistral-7B-Instruct-v0.2"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
baseline = Mistral7B(model=model, tokenizer=tokenizer)
df_base = run_mmlu(model=baseline)

- Avalia o modelo `./mistral-sql-lora-v3` originado por fine-tuning

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig

lora_path = "./mistral-sql-lora-v3"
device = "cuda" if torch.cuda.is_available() else "cpu"
config = PeftConfig.from_pretrained(lora_path)
base_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  device_map="auto",
  torch_dtype="auto"
)
peft_model_v3 = PeftModel.from_pretrained(base_model, lora_path)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

ministral_sql_lora_v3 = Mistral7B(model=model, tokenizer=tokenizer)
df_finetuned = run_mmlu(model=ministral_sql_lora_v3)

- Avalia o modelo `./mistral-sql-lora-v4` originado por fine-tuning

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig

lora_path = "./mistral-sql-lora-v4"
device = "cuda" if torch.cuda.is_available() else "cpu"
config = PeftConfig.from_pretrained(lora_path)
base_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  device_map="auto",
  torch_dtype="auto"
)
peft_model_v3 = PeftModel.from_pretrained(base_model, lora_path)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

ministral_sql_lora_v4 = Mistral7B(model=model, tokenizer=tokenizer)
df_finetuned = run_mmlu(model=ministral_sql_lora_v4)

- Comparação dos modelos base e fine-tuned

In [ ]:
df_base = df_base.rename(columns={"accuracy": "accuracy_base"})
df_finetuned = df_finetuned.rename(columns={"accuracy": "accuracy_finetuned"})

df = pd.merge(df_base, df_finetuned, on=["task", "category"])
df["variation_percent"] = (
    (df["accuracy_finetuned"] - df["accuracy_base"]) / df["accuracy_base"]
) * 100

agg = df.groupby("category").agg({
    "accuracy_base": "mean",
    "accuracy_finetuned": "mean",
    "variation_percent": "mean"
}).reset_index()

agg_total = pd.DataFrame([{
    "category": "Overall",
    "accuracy_base": df["accuracy_base"].mean(),
    "accuracy_finetuned": df["accuracy_finetuned"].mean(),
    "variation_percent": df["variation_percent"].mean()
}])

final_report = pd.concat([agg_total, agg], ignore_index=True)

print("\n=== Variação por Categoria ===\n")
print(final_report.to_markdown(index=False))

print("\n=== Variação por Tarefa ===\n")
print(df[["task", "category", "accuracy_base", "accuracy_finetuned", "variation_percent"]].to_markdown(index=False))